In [ ]:

from matplotlib import pyplot as plt

from src.image.video.snapshot import VideoCamera
from src.image.object_detection.keras_detector import KerasDetector, configure_simple_model
from src.image.video.labelled import LabelledVideo
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
import uuid
from os import listdir
import matplotlib.image as mpimg
import keras
from os.path import isfile, join
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Flatten, Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.applications import densenet, vgg16
from keras.applications.vgg16 import preprocess_input
from src.real_time.matplotlib_backend import MatplotlibLoop


# Image Demo Notebook

This notebook will lead you through multiple tasks dealing with applying deep learning to image recognition tasks. You will learn how to load and save images

Learning objectives:

- 

## 1 Recognizing objects in images/video

In the cell below you can run a program that uses AI to recognize objects in images. Try to see if you can make it recognize some objects around you and perhaps find some objects that it can not detect. 


`Program for object recognition.`

In [ ]:
# In this cell, the user should be able to write the labels from the object recognitions program to a file

### Creating an experiment

1. Using Python in the cell above, make sure the labels from the program are stored to a file.
1. Select a number of object for testing the object recognition program.
1. Run the program while showing it the object you selected. Make sure to show all the objects and to time (approximately) when you showed each of the objects.

### Analysing the experiment

1. In the cell below, reload the file with the experiment data

`Make some kind of usefull visualization, which they can easily access.`

In [ ]:
net = KerasDetector(model_name="mobilenet",exlude_animals=  True)

In [ ]:
plt.close("all")
%matplotlib inline
%matplotlib auto
the_video = LabelledVideo(net,video_length=14 )
the_video.start()
while(not the_video.real_time_backend.stop_now ):
    plt.pause(.5)

## 2 Object classification


### Taking pictures

Run the following cell and take multiple pictures of five objects of your choice after labeling the object

In [ ]:
%matplotlib notebook
num_pictures_each = 2
num_objects = 2
num_labels_predicted = 4

frames = []
labels = []
for i in range(num_objects):
    label_name = input("Object label: ")
    labels.append(label_name)
    back_end = MatplotlibLoop()
    my_camera = VideoCamera(n_photos = num_pictures_each, backend=back_end, title = "Hold object before camera to take picture and press enter")

    my_camera.start()
    while(not my_camera.real_time_backend.stop_now ):
        plt.pause(.5)

    
    frames.append(my_camera.photos[:num_pictures_each])


### Classifying images


We load a neural network to categorize your object pictures

In [ ]:
net = KerasDetector(model_name="inception_resnet_v2",n_labels_returned = num_labels_predicted, exlude_animals=True)

In [ ]:
%matplotlib inline
plt.close("all")

f, axarr = plt.subplots(num_objects, num_pictures_each, figsize=(4*num_pictures_each,3*num_objects))
axarr=np.asarray(axarr) if num_objects ==1 and num_pictures_each==1 else axarr
axarr = np.expand_dims(axarr, axis=0) if num_objects ==1 else axarr
axarr =  np.expand_dims(axarr, axis=-1) if num_pictures_each ==1 else axarr

for i in range (num_objects):
    print(100*"_" +"\nObject {}".format(labels[i]) )
    for j in range(num_pictures_each):
        if j==0:
            axarr[i,j].set_ylabel(labels[i], rotation=0, size='large', labelpad=5*(len(labels[i])))
        axarr[i,j].set_title("Picture {}".format(j))
        probable_labels=net.label_frame(frames[i][j])[0]
        print("Picture {}: ".format(j),", ".join(probable_labels))
        axarr[i,j].imshow(frames[i][j])
        axarr[i,j].tick_params( which='both', labelbottom=False,labelleft = False,length=0)


### Analyzing the results
1. How accurate are the labels for each object? Decide if one of the probable labels accurately describes the object and  count how many images are correct for each object. 

1. Are there differences in the accuracies for the objects? 

1. If yes, why do you think this happens?

### Checking against categories

1. Find a list of the categories used for training the neural network

1. Do the categories reflect the environment you are in? 

1. Are there categories that you would not expect to encounter? 

1. Can you find an object  that you can not find in the categories? 

1. What do you expect to happen when you classify this object? Take pictures of the object and classify them? What does happen?

## 3 Training an ML classifier

In this exercise you will collaborate with four other groups to design and create a classifier for a common object of your choice. The classifier should be able to detect whether the object is in a picture.

- Find four other groups around you that you will work together with

- Agree on a common object that all of you have in your possession (e.g. a pen, glasses, a coffe cup)


### Creating the dataset

- Each of the groups takes ten pictures with their object in the picture and ten pictures without the object in the picture. 
    - Take care that your image is fully within the red frame 
    - Use different backgrounds for the pictures and rotate your object

In [ ]:
%matplotlib inline
%matplotlib auto
#TODO make loop
my_camera = VideoCamera(n_photos = 10,title = "Hold object before camera to take picture and press enter", block = True)
my_camera.start()
while(not my_camera.real_time_backend.stop_now ):
    plt.pause(.5)
positive_frames = my_camera.photos
my_camera = VideoCamera(n_photos = 10,title = "Take pictures without the object by pressing enter", block = True)
while(not my_camera.real_time_backend.stop_now ):
    plt.pause(.5)
negative_frames = my_camera.photos



- Replace FOLDERPATH in the following cell with the full path to the folder where you want your images to be saved
    - On Windows, remember to double all the backslashes

In [ ]:
save_folder = "FOLDERPATH"
save_folder = "/home/lauri/data/test"

### Data augmentation
We can increase the number of training images for the classifier by using our knowledge about images. If an image is flipped or slightly rotated, it will most likely still display the same object but for our classifier look entirely different. 

This can be exploited by adding the flipped image and slightly rotated versions of the same image to the dataset dataset. In our case, this will increase the number of training images ten times while not increasing the amount of work we have to put in.

This  common technique for preprocessing images is called *data augmentation*.


In [ ]:
#  TODO later do augmentation on our own
# right now use keras


datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
num_augmentations = 36
for prefix, cur_list in zip(["pos", "neg"], [positive_frames, negative_frames]):
    i = 0
    for batch in datagen.flow(np.stack(cur_list)[:, :224, :224],save_to_dir=save_folder, save_prefix=prefix, save_format='jpg'):
        i += 1
        if i > num_augmentations:
            break 

- Check the folder you specified in the beginning. You should see many images with and without your object in it.

- Now share your images with the other groups and also add their images to your folder

### Preprocessing the data 
- Load the images and normalize them
- Save the mean and standard deviation used to normalize the image so you can preprocess future images



In [ ]:
# get a list of the positive and negative images 
positive_img_files = [f for f in listdir(save_folder) if isfile(join(save_folder, f)) and "pos" in f]
negative_img_files = [f for f in listdir(save_folder) if isfile(join(save_folder, f)) and "neg" in f]

In [ ]:
#load the images 
positive_imgs = np.stack (mpimg.imread(join(save_folder, file_name)) for file_name in positive_img_files)
neg_imgs = np.stack (mpimg.imread(join(save_folder, file_name)) for file_name in negative_img_files)

In [ ]:
#create your dataset 
x = np.vstack([positive_imgs, neg_imgs])
y = np.hstack([np.ones((len(positive_imgs))), np.zeros((len(neg_imgs)))])
y = to_categorical(y, num_classes= 2)

In [ ]:
x.min()

Right now we simply have a list of images where the first half of them contains your object and the second one does not. We will shuffle the list, so that they are more randomly distributed.

In [ ]:
random_permuation = np.random.permutation(len(y))
x = x[random_permuation]
y= y[random_permuation]
x = preprocess_input(x)
x_train = x[:int(0.8*len(x))]
y_train = y[:int(0.8*len(x))]
x_test = x[int(0.8*len(x)): int(0.9*len(x)) ]
y_test = y[int(0.8*len(x)): int(0.9*len(x)) ]
x_val = x[int(0.9*len(x)):]
y_val = y[int(0.9*len(x)):]

In [ ]:
x.max()

### Training the classifier

Now we are ready to create the classifier and train it. We will use a pre-trained neural network and fine-tune it with our dataset.

In [ ]:
base_model = vgg16.VGG16(input_shape=(224, 224,3),include_top = False)

In [ ]:
w = Flatten()(base_model.output)
w = Dense(1024, activation='relu')(w)
w = Dropout(0.5)(w)

predictions = Dense(2, activation = 'softmax')(w)

#create graph of your new model
head_model = Model(input = base_model.input, output = predictions)


head_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
head_model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))